# Измерение времени выполнения. Профилирование кода

IPython имеет "магические" команды для профилировки:
Встроенные в IPython:
- `%time`: Время выполнения выражения (строки)
- `%timeit`: Время выполнения выражения, оцененное по многократным запускам

Из расширения `line_profiler`:
- `%prun`: Запуск кода с профилированием по командам
- `%lprun`: Запуск кода с профилированием по строкам

Из расширения `memory_profiler`:
- `%memit`: Использование памяти для выражения
- `%mprun`: Использование памяти с профилированием по строкам



## Команды %timeit and %time

Аналогично другим "магическим" командам (см. [IPython Magic Commands](01.03-Magic-Commands.ipynb) ) символ %% дает возможность измерения для блока кода.

`%timeit` для строки кода

`%%timeit` для блока кода

Команда сама решает сколько раз повторять выполнение кода. Для быстрых вычислений - побольше, для долгих - поменьше.

In [1]:
%timeit sum(range(100))

1.67 µs ± 251 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [2]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

341 ms ± 8.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Повторение вычислений может привести к неправильной оценке времени. Например, сортировка не отсортированного списка и отсортированного занимает разное время.

In [3]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

832 µs ± 169 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Для таких случаев лучше использовать `%time` - измеряет время однократного выполнения кода .

In [4]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
CPU times: user 20.1 ms, sys: 5 µs, total: 20.1 ms
Wall time: 20.1 ms


In [5]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
CPU times: user 2.76 ms, sys: 0 ns, total: 2.76 ms
Wall time: 2.77 ms


Предварительно отсортированный список сортируется быстрее.

Однако обратите внимение на измеренное время: `%time` показывает гораздо большее время, чем `%timeit`.

`%time` меряет полное время выполнения, включая все подготовительные действия, работу сборшика мусора и т.п.

Аналогично работает `%%time` для блока кода.

In [6]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

CPU times: user 406 ms, sys: 9 µs, total: 406 ms
Wall time: 407 ms


## Профилирование скриптов и функций: %prun

Создадим функцию для тестов и профилируем ее `%prun`

In [7]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

In [8]:
%prun sum_of_lists(1000000)

## Профилирование по строкам %lprun

`%prun` профилирует по командам, но бывает удобней смотреть на профилирование по строкам кода.  Такая возможность есть в команде `%lprun` из расширения `line_profiler`.

Сторонние расширения подключаются "магической" командой `%load_ext`.

In [12]:
!pip install line_profiler

In [10]:
%load_ext line_profiler

Для `%lprun` указываем название функции для профилирования и сам код:

In [11]:
%lprun -f sum_of_lists sum_of_lists(5000)

Теперь информация представлена для каждой строки кода в профилируемой функции.

Показано время выполнения, число вызовов и другая информация для всех выполняемых команд.

_Вопрос: почему команда for вызывается 6 раз, а команды в теле цикла только 5?_  

## Профилирование памяти: %memit and %mprun

Кроме времени выполнения важная информация содержится в количестве используемой памяти. Профилирование памяти реализовано в расширении `memory_profiler`.

In [13]:
!pip install memory_profiler

In [14]:
%load_ext memory_profiler

Команда `%memit` по синтаксису похожа  на `%timeit` , возвращает используемую память.

Команда `%mprun` (похожа на `%lprun`) профилирует память по строкам.

In [15]:
%memit sum_of_lists(1000000)

peak memory: 170.20 MiB, increment: 62.83 MiB


`%mprun` работает только для отдельных скриптов (файлов).

Создадим с помощью команды `%%file` файл-код `mprun_demo.py` содержащий функцию `sum_of_lists` и профилируем ее.

In [16]:
# создаем файл с кодом
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Writing mprun_demo.py


In [17]:
# профилируем ~ 5 мин
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(100000)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/memory_profiler.py", line 847, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/memory_profiler.py", line 850, in disable
    sys.settrace(self._original_trace_function)



Показаны данные по потреблении памяти, в том числе изменение памяти от для каждой строки. Обратите внимание, что оболочка Ipython также тратит некоторое количество памяти.

# CPU vs GPU

Мы знаем, что сегодня графические ускорители GPU гораздо мощнее чем центральные процессоры CPU. Сравним их работу.

_Примечание: в тетрадке должна быть включена поддержка GPU (см. "Среда выполнения")_

Утилита [pynvml](https://pypi.org/project/pynvml/) - позволяет управлять и наблюдать за ресурсами GPU.

_Примечание: После установки перезапустите среду выполнения._  

In [18]:
# нужно перезапустить среду
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00


Создадим какую-нибудь нейронную сеть и измерим время обучения ее на CPU и GPU доступных в Colab.

In [1]:
import timeit
import torch
import torch.nn as nn
import torchvision.models as models
model = models.vgg19() # какая-то сеть

#  обучение модели на выбранном устройстве
def train(model, input_size, output_size, optimizer, loss_fn, epochs, device):
  model.requires_grad_(True) # автодифференцирование вкл.
  model.to(device) # модель передаем на выбранное устройство
  model.train() # переводим в режим обучения
  for _ in range(epochs): # цикл обучения
    input = torch.rand(input_size, device=device) # вход
    target = torch.rand(output_size, device=device) # желаемый выход
    optimizer.zero_grad() # метод обучения
    output = model(input) # считаем выход модели
    loss = loss_fn(output, target) # считаем функцию ошибки
    loss.backward() # считаем производные
    optimizer.step() # изменяем параметры

# измерение времени
def benchmark(device):
  batch_size = 16
  channels = 3
  input_width = 224
  input_height = 224
  classes = 1000
  input_size = (batch_size, channels, input_width, input_height)
  output_size = (batch_size, classes)
  train(model, input_size, output_size,
            optimizer=torch.optim.SGD(model.parameters(),
                                      lr=1e-2, momentum=0.9),
            loss_fn=nn.CrossEntropyLoss(),
            epochs=2, device=device)


Команда timeit утилиты [timeit](https://docs.python.org/3/library/timeit.html) позволяет замерить время (python интерфейс для timeit).

In [3]:
timeit.timeit('benchmark(torch.device("cpu"))', number=1, globals=globals()) # 1 повтор, на cpu, примерно  90 сек


72.35343039700001

In [8]:
timeit.timeit('benchmark(torch.device("cuda"))', number=1, globals=globals()) # 1 повтор, на gpu, примерно  3 сек

0.01985058000002482

Информация о GPU:

In [9]:
torch.cuda.get_device_name() # название

'Tesla T4'

In [10]:
torch.cuda.get_device_properties(torch.cuda.current_device()) # общие характеристики

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)

In [11]:
# информация о используемой GPU памяти
#import torch
print('Memory usage:', torch.cuda.memory_usage(torch.cuda.current_device()))
print('Utilization:', torch.cuda.utilization(torch.cuda.current_device()))
print('GPU Processes:', torch.cuda.list_gpu_processes(torch.cuda.current_device()))
print('Mem info:', torch.cuda.mem_get_info(torch.cuda.current_device()))
print('Memory stats:', torch.cuda.memory_stats(torch.cuda.current_device()))
print('Memory summary:', torch.cuda.memory_summary(torch.cuda.current_device()))

Memory usage: 0
Utilization: 0
GPU Processes: GPU:0
process      32673 uses     4266.000 MB GPU memory
Mem info: (11359289344, 15835660288)
Memory stats: OrderedDict([('active.all.allocated', 2078), ('active.all.current', 78), ('active.all.freed', 2000), ('active.all.peak', 124), ('active.large_pool.allocated', 1367), ('active.large_pool.current', 32), ('active.large_pool.freed', 1335), ('active.large_pool.peak', 64), ('active.small_pool.allocated', 711), ('active.small_pool.current', 46), ('active.small_pool.freed', 665), ('active.small_pool.peak', 75), ('active_bytes.all.allocated', 147649585152), ('active_bytes.all.current', 1169131520), ('active_bytes.all.freed', 146480453632), ('active_bytes.all.peak', 3296273408), ('active_bytes.large_pool.allocated', 147564986368), ('active_bytes.large_pool.current', 1166934016), ('active_bytes.large_pool.freed', 146398052352), ('active_bytes.large_pool.peak', 3292856320), ('active_bytes.small_pool.allocated', 84598784), ('active_bytes.small_poo

# Задания
* Научится профилировать произвольный код
* Строить графики зависимости времени\памяти от размера массивов или моделей.

# Ссылки
Использованы и адаптированы материалы:
* https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.07-Timing-and-Profiling.ipynb

* https://adrianstoll.com/post/colab-cpu-vs-gpu-benchmarks/

см. также
* Профилирование в pytorch: https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html

* Модуль sys: https://docs-python.ru/standart-library/modul-sys-python/funktsija-getsizeof-modulja-sys/